In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
from pyproj import CRS, Transformer

from web_analytics import Schedules, make_isochrones
from web_analytics.utils import add_travel_times, merge_schedules

In [ ]:
place = "Madrid, Spain"
center_point = (-3.70331, 40.41688)  # lon, lat (EPSG 4326)
trip_times = [5, 10, 15, 30, 60, 90]  # min
travel_speed = 4.5  # km/h

In [ ]:
G = ox.graph_from_place(place, network_type="walk")
G = ox.project_graph(G)
add_travel_times(G, travel_speed)

In [ ]:
schedules = Schedules()
merge_schedules(G, nx.MultiDiGraph(nx.union_all(schedules.get_all_trip_graphs().values())))

In [ ]:
transformer = Transformer.from_crs(CRS.from_epsg(4326), G.graph["crs"], always_xy=True)
center_point_proj = transformer.transform(*center_point)
center_node = ox.nearest_nodes(G, center_point_proj[0], center_point_proj[1])

In [ ]:
isochrone_gdf = make_isochrones(G, trip_times, [center_node])

In [ ]:
fig, ax = ox.plot_graph(
    G, show=False, close=False, edge_color="#999999", edge_alpha=0.2, node_size=0
)
isochrone_gdf.plot(
    ax=ax, column="trip_time", cmap="plasma_r", categorical=True, ec="none", alpha=0.6, zorder=-1
)
plt.show()

In [ ]:
gdf_explore = isochrone_gdf.copy()
gdf_explore.sort_values("trip_time", inplace=True)
gdf_explore.reset_index(drop=True, inplace=True)
gdf_explore["geom_diff"] = None
gdf_explore.at[0, "geom_diff"] = gdf_explore.at[0, "geometry"]

for index in range(1, len(gdf_explore)):
    gdf_explore.at[index, "geom_diff"] = gdf_explore.at[index, "geometry"].difference(
        gdf_explore.at[index - 1, "geometry"]
    )

gdf_explore["geometry"] = gdf_explore["geom_diff"]
gdf_explore.drop("geom_diff", axis=1, inplace=True)

In [ ]:
gdf_explore.explore(column="trip_time", cmap="plasma_r", categorical=True)